# Deel 1

Santa wil zijn elfjes speelgoed laten maken, maar hij heeft zijn administratie nog niet op orde. Hij heeft een lijst van onderdelen die in elk soort speelgoed zitten. Het probleem is alleen dat sommige onderdelen weer uit verdere onderdelen bestaan, wat het tellen van het aantal onderdelen moeilijk maakt.

Stel bijvoorbeeld dat hij deze lijst zou krijgen (de missende onderdelen kan je voor nu negeren): 

    46 onderdelen missen
    Zoink: 9 Oink, 5 Dink
    Floep: 2 Flap, 4 Dink
    Flap: 4 Oink, 3 Dink
    
In dit voorbeeld zijn Zoinks makkelijk: er zitten in totaal 14 (9+5) onderdelen in. Een Floep is lastiger, omdat de Flappen die erin zitten elk ook uit meerdere onderdelen bestaan. Elke Flap bestaat uit 7 onderdelen. In een Floep zitten dus 18 (2*7+4) onderdelen. 

 Gegeven de speelgoedlijst, vind het stuk speelgoed met het grootste aantal onderdelen. Dit aantal is vervolgens het antwoord op deel 1.

Vul hieronder het aantal onderdelen in. 

In [1]:
file = 'speelgoedlijst.txt'
# file = 'minilijst.txt' # Uncomment om te testen met een andere bestand 

# lees het tekstbestand 
with open(file,'r') as f:
    line1 = f.readline()
    lines = f.read().splitlines()

# aantal ontbrekende onderdelen van de eerste regel 
total = int(line1.split()[0])

# maakt een Python dictionary van de speelgoedlijst
comps_dic = {}
for line in lines:
    key,tail = line.split(':')
    comps_dic[key] = {}
    for comp in tail.split(','):
        amt,k = comp.split()
        comps_dic[key][k] = int(amt)

Met deze code hebben we de lijst geladen als een geneste "Python-dictionary". De "minilijst" levert op bijvoorbeeld het dictionary 

    {'Zoink': {'Oink': 9, 'Dink': 5},
     'Floep': {'Flap': 2, 'Dink': 4},
     'Flap': {'Oink': 4, 'Dink': 3}}

Het aantal flappen in een floep (namelijk 2) kan worden verkregen via `comps_dic['Floep']['Flap']`.

In het volgende deel lossen we het probleem op door dit dictionary te doorlopen. We maken twee niewe dictionary's: `count_dic`, dat het totale aantal stukken in elk item op de lijst geeft, en `top_dic`, dat zegt of elk item aan de "top" van de hiërarchie staat (of het item geen onderdeel is). `top_dic` wordt gebruikt voor het tweede deel van de puzzel. Voor elk item in de lijst gaat de functie `get_count()` door elk van zijn onderdelen en telt het totale bedrag. Als het opgegeven onderdeel niet op de hoofdlijst voorkomt, tellen we het aantal gewoon op bij het huidige aantal onderdelen. Anders telt de functie eerst het aantal onder-onderdelen in het onderdeel in een recursieve aanroep, en het bedrag dat moet worden toegevoegd aan het aantal onderdelen wordt vermenigvuldigd met het aantal onder-onderdelen. In dit tweede geval merkt de functie ook op dat het onderdeel niet bovenaan de hiërarchie staat.

Om overbodig werk te voorkomen, stopt de functie als het item al is geteld (d.w.z. als de bijbehorende telling al niet nul is). 

In [2]:
counts_dic = {key:0 for key in comps_dic}
top_dic = {key:True for key in comps_dic}

def get_counts(key):
    global comps_dic
    global counts_dic
    if not counts_dic[key]: #equivalent - if key2 in comps_dic != 0:
        count = 0
        for key2 in comps_dic[key]:
            k_count = comps_dic[key][key2]
            if key2 in comps_dic:
                top_dic[key2] = False
                get_counts(key2)
                k_count *= counts_dic[key2]
            count += k_count
        counts_dic[key] = count

for key in comps_dic:
    get_counts(key)

print('grootste aantal onderdelen:',max([counts_dic[k] for k in comps_dic]))

grootste aantal onderdelen: 293867


# Deel 2

Terwijl je bezig was met het tellen van de onderdelen, waren een paar ijverige elfjes al begonnen! Ze hebben speelgoed in elkaar gezet en ingepakt, maar zijn alweer vergeten wat erin zat. Gek genoeg weten ze nog wel hoeveel onderdelen ze hebben gebruikt.

De elfjes hebben alleen speelgoed ingepakt, en geen onderdelen zoals accu's of ijzer. Blijkbaar is er niemand stout geweest dit jaar.

In het eerdere voorbeeld waren er 46 missende onderdelen. Stel dat er 3 cadeaus al ingepakt waren. Zoinks en Floepen zijn hier de mogelijke stukken speelgoed, want ze worden niet als onderdelen gebruikt. Er is maar 1 manier om met alle missende onderdelen onderdelen precies 3 stukken speelgoed te maken: 2 Zoinks en 1 Floep (14+14+18=46).

Als je de stukken speelgoed weet, kan je je antwoord vinden door de eerste letters van het speelgoed op alfabetische volgorde te zetten. In het bovenstaande voorbeeld zou dat dus `FZZ` worden.

Er zijn al 20 cadeaus ingepakt. Gegeven het aantal missende onderdelen in de speelgoedlijst, wat zijn dan de beginletters (op alfabetische volgorde)?

In [3]:
heads_dic = {counts_dic[key]:key[0] for key in filter(lambda x: top_dic[x], comps_dic)}
counts = [counts_dic[key] for key in filter(lambda x: top_dic[x], comps_dic)]
counts.sort(reverse = True)
print(counts) # Lijst van de "top" artikelbedragen, in aflopende volgorde 
print(heads_dic) # Lijst met de eerste letters die bij elk bedrag horen 

[293867, 260702, 12649, 11082, 6457, 1614, 737, 359]
{1614: 'L', 11082: 'H', 12649: 'E', 260702: 'Q', 737: 'P', 359: 'T', 293867: 'B', 6457: 'D'}


Nadat we het bedrag voor elk "top" item hebben gekregen, lossen we de puzzel op met behulp van de `get_wgts`-functie die hieronder is gedefinieerd. De functie begint met het nemen van zoveel mogelijk van het eerste item (het item met de meeste onderdelen) en zoekt in een recursieve aanroep naar een verzameling met de resterende items die het resterende aantal items (van de in totaal 20 vereist) en het resterende aantal onderdelen. Als er geen oplossing wordt gevonden, wordt het bedrag van het eerste item verlaagd en wordt het opnieuw geprobeerd.

Door elke keer zoveel mogelijk van het item met de meeste stuks te nemen, zorgen we ervoor dat de collecties met het juiste aantal stuks worden verkregen in een volgorde waarin het aantal gebruikte items toeneemt.

In [5]:
def get_wgts(arr,tot,wgtsum): 
    # gaat ervan uit dat de lijst in aflopende volgorde is gesorteerd 
    # retourneert: vlag voor of er een oplossing bestaat en een lijst met bedragen voor elk item
    
    if not arr:                            #equivalent - if arr == []:
        return (tot==0 and wgtsum==0),[]
    exists = False
    for w in range(min(1+wgtsum,1+tot//arr[0]))[::-1]:
        exists,sol = get_wgts(arr[1:],tot-w*arr[0],wgtsum-w)
        if exists:
            return True, [w] + sol
    return False,[]

flag,sol = get_wgts(counts,total,20)
print('Oplossing:')
for i in range(len(sol)):
    print(heads_dic[counts[i]],sol[i],end = '; ')
print()
print('Totaal aantal cadeaus:',sum(sol))

sol_arr = []
for i in range(len(sol)):
    sol_arr += [heads_dic[counts[i]]]*sol[i]
sol_arr.sort()
print('Oplossing string:',''.join(sol_arr))

Oplossing:
B 2; Q 3; E 3; H 3; D 2; L 1; P 3; T 3; 
Totaal aantal cadeaus: 20
Oplossing string: BBDDEEEHHHLPPPQQQTTT
